1차

In [ ]:
# ============================================================
# 0. 기본 환경 설정
# ============================================================
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


import os
import cv2
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np



def safe_load_image(path):
    """고속 로딩 + 깨진 JPEG 자동 처리"""
    try:
        img = cv2.imread(path)
        if img is None:
            img = Image.open(path).convert("RGB")
            img = np.array(img)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img
    except Exception:
        return None


class FastHappyDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.paths = []
        self.labels = []
        self.transform = transform

        for emo in os.listdir(root_dir):
            emo_dir = os.path.join(root_dir, emo)
            if not os.path.isdir(emo_dir):
                continue

            label = 1 if emo == "happy" else 0

            for f in os.listdir(emo_dir):
                if f.lower().endswith(("jpg", "jpeg", "png")):
                    self.paths.append(os.path.join(emo_dir, f))
                    self.labels.append(label)

        print(f"Loaded {len(self.paths)} images from {root_dir}")

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        label = self.labels[idx]

        img = safe_load_image(path)
        if img is None:
            return self.__getitem__((idx + 1) % len(self.paths))

        if self.transform:
            img = self.transform(image=img)["image"]

        return img, torch.tensor(label).long()


# ============================================================
# 1. Albumentations Transform
# ============================================================
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])


# ============================================================
# 2. Dataset & DataLoader
# ============================================================
train_root = "/workspace/merge_data/data/img/train"
val_root   = "/workspace/merge_data/data/img/val"

train_dataset = FastHappyDataset(train_root, transform=transform)
val_dataset   = FastHappyDataset(val_root, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=4,
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=4,
)


# ============================================================
# 3. MobileViT-S 모델 불러오기
# ============================================================
from torchvision.models import mobilevit_s

model = mobilevit_s(weights="IMAGENET1K_V1")  # pretrained
model.classifier = nn.Sequential(
    nn.Linear(640, 2)   # MobileViT-S output dim = 640
)

model = model.to(device)
print("Model loaded on:", next(model.parameters()).device)


# ============================================================
# 4. Optimizer & Loss
# ============================================================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-5, weight_decay=1e-5)

scaler = torch.amp.GradScaler("cuda")


# ============================================================
# 5. 학습 함수
# ============================================================
def train_one_epoch():
    model.train()
    total_loss, total_correct, total_samples = 0, 0, 0

    for imgs, labels in tqdm(train_loader, desc="Train"):
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast("cuda"):
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        preds = outputs.argmax(dim=1)
        total_correct += (preds == labels).sum().item()
        total_loss += loss.item() * labels.size(0)
        total_samples += labels.size(0)

    return total_loss / total_samples, total_correct / total_samples


# ============================================================
# 6. 검증 함수
# ============================================================
def validate():
    model.eval()
    total_loss, total_correct, total_samples = 0, 0, 0

    with torch.inference_mode(), torch.amp.autocast("cuda"):
        for imgs, labels in tqdm(val_loader, desc="Validation"):
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            preds = outputs.argmax(dim=1)

            total_correct += (preds == labels).sum().item()
            total_loss += loss.item() * labels.size(0)
            total_samples += labels.size(0)

    return total_loss / total_samples, total_correct / total_samples


# ============================================================
# 7. 전체 학습 루프
# ============================================================
num_epochs = 10
best_acc = 0

for epoch in range(num_epochs):
    print(f"\n===== Epoch {epoch+1}/{num_epochs} =====")

    train_loss, train_acc = train_one_epoch()
    val_loss, val_acc     = validate()

    print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "mobilevit_s_fastloader_best.pth")
        print("💾 Best model saved!")

print(f"\n🎉 Training Completed — Best Val Acc: {best_acc:.4f}")


# ============================================================
# 8. 평가 성능 출력
# ============================================================
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

model.eval()
all_preds, all_labels = [], []

with torch.inference_mode(), torch.amp.autocast("cuda"):
    for imgs, labels in tqdm(val_loader, desc="Final Evaluation"):
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

acc  = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, zero_division=0)
rec  = recall_score(all_labels, all_preds, zero_division=0)
f1   = f1_score(all_labels, all_preds, zero_division=0)

print("\n===== Final Metrics =====")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, digits=4))

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Not Happy (0)", "Happy (1)"],
            yticklabels=["Not Happy (0)", "Happy (1)"])
plt.title("Confusion Matrix")
plt.show()


2차

In [ ]:
import os
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)
from tqdm import tqdm
from transformers import (
    MobileViTImageProcessor,
    MobileViTForImageClassification,
)

# =====================================================
# 1. SEED 고정
# =====================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"


# =====================================================
# 2. DATASET
# =====================================================
class HappyDataset(Dataset):
    def __init__(self, root_dir, processor):
        self.root_dir = root_dir
        self.processor = processor
        self.samples = []

        classes = sorted(os.listdir(root_dir))  # ["happy", "other"]
        self.class_to_idx = {c: i for i, c in enumerate(classes)}

        for cls in classes:
            folder = os.path.join(root_dir, cls)
            for file in os.listdir(folder):
                if file.lower().endswith(("jpg", "jpeg", "png")):
                    self.samples.append(
                        (os.path.join(folder, file), self.class_to_idx[cls])
                    )

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")

        processed = self.processor(images=img, return_tensors="pt")
        pixel_values = processed["pixel_values"].squeeze(0)

        return pixel_values, torch.tensor(label)


# =====================================================
# 3. LOAD Processor & Model  ← MobileViT
# =====================================================
processor = MobileViTImageProcessor.from_pretrained(
    "apple/mobilevit-small"
)

model = MobileViTForImageClassification.from_pretrained(
    "apple/mobilevit-small",
    num_labels=2,
    ignore_mismatched_sizes=True
).to(device)


# =====================================================
# 4. PATHS
# =====================================================
train_root = "/workspace/merge_data_binary/new_data/img_binary"
val_root   = "/workspace/merge_data_binary/new_data_val/img_binary"

train_dataset = HappyDataset(train_root, processor)
val_dataset   = HappyDataset(val_root, processor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8)


# =====================================================
# 5. Optimizer
# =====================================================
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)


# =====================================================
# 6. TRAINING LOOP
# =====================================================
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0

    pbar = tqdm(train_loader, desc=f"[Epoch {epoch+1}] Training")
    for pixel_values, labels in pbar:
        pixel_values = pixel_values.to(device)
        labels = labels.to(device)

        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix({"loss": loss.item()})

    avg_loss = total_loss / len(train_loader)
    print(f"\n[Epoch {epoch+1}] Train Loss: {avg_loss:.4f}")

    # -------------------------
    # Validation
    # -------------------------
    model.eval()
    preds, gts = [], []

    with torch.no_grad():
        for pixel_values, labels in tqdm(val_loader, desc="Validating"):
            pixel_values = pixel_values.to(device)
            outputs = model(pixel_values=pixel_values)

            pred = outputs.logits.argmax(dim=1).cpu()
            preds.extend(pred.tolist())
            gts.extend(labels.tolist())

    # ============================
    # METRICS
    # ============================
    acc = accuracy_score(gts, preds)
    prec = precision_score(gts, preds, average="binary")
    rec = recall_score(gts, preds, average="binary")
    f1 = f1_score(gts, preds, average="binary")
    cm = confusion_matrix(gts, preds)

    print("\n========== METRICS ==========")
    print(f"Accuracy  : {acc:.4f}")
    print(f"Precision : {prec:.4f}")
    print(f"Recall    : {rec:.4f}")
    print(f"F1-Score  : {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print("============================\n")


3차

In [1]:
import os
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)
from tqdm import tqdm
from transformers import (
    MobileViTImageProcessor,
    MobileViTForImageClassification,
)
from torch import amp


# =====================================================
# 1. SEED
# =====================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"


# =====================================================
# 2. DATASET (4-class)
# =====================================================
class EmotionDataset(Dataset):
    def __init__(self, root_dir, processor):
        self.root_dir = root_dir
        self.processor = processor
        self.samples = []

        self.valid_folders = ["happy", "anger", "panic", "sadness"]
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.valid_folders)}

        for cls in self.valid_folders:
            folder_path = os.path.join(root_dir, cls)
            if not os.path.exists(folder_path):
                continue

            label = self.class_to_idx[cls]

            for file in os.listdir(folder_path):
                if file.lower().endswith(("jpg", "jpeg", "png")):
                    self.samples.append((os.path.join(folder_path, file), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")

        processed = self.processor(images=img, return_tensors="pt")
        pixel_values = processed["pixel_values"].squeeze(0)
        return pixel_values, torch.tensor(label)


# =====================================================
# 3. LOAD Model  ← ★ MobileViT Small
# =====================================================
processor = MobileViTImageProcessor.from_pretrained(
    "apple/mobilevit-small"
)

model = MobileViTForImageClassification.from_pretrained(
    "apple/mobilevit-small",
    num_labels=4,
    ignore_mismatched_sizes=True
).to(device)

scaler = amp.GradScaler(device="cuda")


# =====================================================
# 4. PATHS
# =====================================================
train_root = "/workspace/merge_data/data/img/train"
val_root   = "/workspace/merge_data/data/img/val"

train_dataset = EmotionDataset(train_root, processor)
val_dataset   = EmotionDataset(val_root, processor)

train_loader = DataLoader(
    train_dataset, batch_size=32, shuffle=True,
    num_workers=8, pin_memory=True, persistent_workers=True
)
val_loader = DataLoader(
    val_dataset, batch_size=32, shuffle=False,
    num_workers=8, pin_memory=True, persistent_workers=True
)


# =====================================================
# 5. Optimizer
# =====================================================
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)


# =====================================================
# 6. TRAINING LOOP
# =====================================================
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0

    pbar = tqdm(train_loader, desc=f"[Epoch {epoch+1}] Training")

    for pixel_values, labels in pbar:
        pixel_values = pixel_values.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()

        # AMP
        with amp.autocast("cuda"):
            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        pbar.set_postfix({"loss": loss.item()})

    avg_loss = total_loss / len(train_loader)
    print(f"\n[Epoch {epoch+1}] Train Loss: {avg_loss:.4f}")

    # -------------------------
    # Validation
    # -------------------------
    model.eval()
    preds, gts = [], []

    with torch.no_grad():
        for pixel_values, labels in tqdm(val_loader, desc="Validating"):
            pixel_values = pixel_values.to(device, non_blocking=True)

            with amp.autocast("cuda"):
                outputs = model(pixel_values=pixel_values)
                pred = outputs.logits.argmax(dim=1).cpu()

            preds.extend(pred.tolist())
            gts.extend(labels.tolist())

    # ★ 4-class 지표
    acc = accuracy_score(gts, preds)
    prec = precision_score(gts, preds, average='macro')
    rec  = recall_score(gts, preds, average='macro')
    f1   = f1_score(gts, preds, average='macro')
    cm   = confusion_matrix(gts, preds)

    print("\n========== METRICS ==========")
    print(f"Accuracy  : {acc:.4f}")
    print(f"Precision : {prec:.4f}")
    print(f"Recall    : {rec:.4f}")
    print(f"F1-Score  : {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print("============================\n")


Some weights of MobileViTForImageClassification were not initialized from the model checkpoint at apple/mobilevit-small and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 640]) in the checkpoint and torch.Size([4, 640]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[Epoch 1] Training: 100%|██████████| 183/183 [01:43<00:00,  1.76it/s, loss=1.33]



[Epoch 1] Train Loss: 1.3666


Validating: 100%|██████████| 36/36 [00:22<00:00,  1.58it/s]



========== METRICS ==========
Accuracy  : 0.4796
Precision : 0.4881
Recall    : 0.4796
F1-Score  : 0.4403
Confusion Matrix:
[[203  22  46  28]
 [ 59  43 161  28]
 [ 38  12 222   8]
 [ 87  23  87  84]]



[Epoch 2] Training: 100%|██████████| 183/183 [01:33<00:00,  1.96it/s, loss=0.876]


...


========== METRICS ==========
Accuracy  : 0.7524
Precision : 0.7595
Recall    : 0.7510
F1-Score  : 0.7512
Confusion Matrix:
[[264  15   9  11]
 [ 13 194  25  59]
 [  8  78 176  18]
 [ 11  31   7 232]]



[Epoch 10] Training: 100%|██████████| 183/183 [01:36<00:00,  1.90it/s, loss=0.651] 



[Epoch 10] Train Loss: 0.2617


Validating: 100%|██████████| 36/36 [00:21<00:00,  1.64it/s]


========== METRICS ==========
Accuracy  : 0.7541
Precision : 0.7581
Recall    : 0.7526
F1-Score  : 0.7537
Confusion Matrix:
[[264  14   9  12]
 [ 15 200  33  43]
 [ 11  67 187  15]
 [ 13  42   9 217]]



In [4]:
!pip install albumentations opencv-python scikit-learn tqdm
!pip install timm

4단계 seg

In [6]:
import os
import json
import random
import numpy as np
import cv2 

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets
from tqdm import tqdm
import copy
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2 

# ★ timm 임포트 추가 ★
import timm

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# ================================
# 3. Transform / Dataset / DataLoader
# ================================
train_transform = A.Compose([
    A.Resize(224, 224), # MobileViT는 256x256이 원래 스펙이지만 224도 가능
    A.HorizontalFlip(p=0.5),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])

data_root = "/workspace/user4/segmented_output_bisenet"

train_full_dataset = datasets.ImageFolder(os.path.join(data_root, "train"), transform=None) 
val_full_dataset = datasets.ImageFolder(os.path.join(data_root, "val"), transform=None) 

target_classes = ["anger", "happy", "panic", "sadness"]

def filter_and_remap_dataset(full_dataset, transform, target_classes): 
    new_samples = []
    for path, class_idx in full_dataset.samples:
        class_name = full_dataset.classes[class_idx]
        if class_name not in target_classes:
            continue
        # 0: OTHERS, 1: HAPPY
        binary_label = 1 if class_name == "happy" else 0
        new_samples.append((path, binary_label))
        
    class CustomBinaryDataset(Dataset):
        def __init__(self, samples, transform):
            self.samples = samples
            self.transform = transform
        def __len__(self):
            return len(self.samples)
        def __getitem__(self, idx):
            path, label = self.samples[idx]
            image = cv2.imread(path)
            if image is None: raise FileNotFoundError(f"Image not found: {path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if self.transform:
                augmented = self.transform(image=image)
                image = augmented['image']
            return image, torch.tensor(label, dtype=torch.long)

    print(f"Original {len(full_dataset)} -> {len(new_samples)} samples.")
    return CustomBinaryDataset(new_samples, transform) 

train_dataset = filter_and_remap_dataset(train_full_dataset, train_transform, target_classes) 
val_dataset = filter_and_remap_dataset(val_full_dataset, val_transform, target_classes) 

batch_size = 32
num_workers = 4 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

print("\n✅ 전처리 및 데이터 로드 완료.")

# ================================
# 4. 모델 정의 (timm 사용: MobileViT)
# ================================
print("\nLoading MobileViT Small model using timm...")

# ★ timm을 사용하면 버전 에러 없이 안전하게 로드 가능합니다 ★
# num_classes=2를 넣으면 마지막 레이어도 자동으로 맞춰줍니다.
model = timm.create_model('mobilevit_s', pretrained=True, num_classes=2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

# ================================
# 5. 학습 함수
# ================================
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        # Train
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # Val
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0

        with torch.inference_mode():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = val_corrects.double() / val_total
        print(f"Val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "best_mobilevit_timm.pth")
            print(">> best model updated")

    print(f"\nBest val Acc: {best_val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model

# ================================
# 6. 학습 실행
# ================================
num_epochs = 10
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs)

# ================================
# 7. 성능 지표
# ================================
model.eval()
all_labels = []
all_preds = []

with torch.inference_mode():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

print("\n=== Val Metrics (1=HAPPY, 0=OTHERS) ===")
print(f"Accuracy : {accuracy_score(all_labels, all_preds):.4f}")
print(f"Precision: {precision_score(all_labels, all_preds, pos_label=1):.4f}")
print(f"Recall   : {recall_score(all_labels, all_preds, pos_label=1):.4f}")
print(f"F1-score : {f1_score(all_labels, all_preds, pos_label=1):.4f}")
print("\nConfusion Matrix:\n", confusion_matrix(all_labels, all_preds))
print("\nReport:\n", classification_report(all_labels, all_preds, target_names=["OTHERS", "HAPPY"]))

device: cuda
Original 9535 -> 9535 samples.
Original 1149 -> 1149 samples.

✅ 전처리 및 데이터 로드 완료.

Loading MobileViT Small model using timm...

Epoch 1/10
------------------------------


100%|██████████| 298/298 [02:36<00:00,  1.90it/s]

Train Loss: 0.2514 Acc: 0.9181


Val   Loss: 0.1219 Acc: 0.9626
>> best model updated

Epoch 2/10
------------------------------


100%|██████████| 298/298 [01:30<00:00,  3.29it/s]

...

Train Loss: 0.0114 Acc: 0.9961


Val   Loss: 0.1267 Acc: 0.9704

Epoch 10/10
------------------------------


100%|██████████| 298/298 [01:18<00:00,  3.81it/s]

Train Loss: 0.0124 Acc: 0.9959


Val   Loss: 0.1263 Acc: 0.9695

Best val Acc: 0.9782

=== Val Metrics (1=HAPPY, 0=OTHERS) ===
Accuracy : 0.9782
Precision: 0.9477
Recall   : 0.9699
F1-score : 0.9587

Confusion Matrix:
 [[834  16]
 [  9 290]]

Report:
               precision    recall  f1-score   support

      OTHERS       0.99      0.98      0.99       850
       HAPPY       0.95      0.97      0.96       299

    accuracy                           0.98      1149
   macro avg       0.97      0.98      0.97      1149
weighted avg       0.98      0.98      0.98      1149



In [16]:
!pip install timm

In [4]:
import timm # ★ timm 라이브러리 임포트

# ================================
# 4. 모델 정의 (timm을 사용한 MobileViT Small)
# ================================
print("Loading MobileViT Small Model via timm...")

# timm을 사용하면 복잡한 레이어 교체 없이 num_classes 인자만으로 처리 가능합니다.
# 'mobilevit_s': MobileViT Small 모델
# pretrained=True: ImageNet 가중치 사용
# num_classes=2: Happy vs Others (2진 분류)
try:
    model = timm.create_model('mobilevit_s', pretrained=True, num_classes=2)
    print("✅ MobileViT loaded via timm")
except ImportError:
    print("🚨 timm 라이브러리가 설치되지 않았습니다. '!pip install timm'을 실행해주세요.")
    # 비상시 ResNet으로 대체 (코드 실행이 멈추지 않게)
    print(">> Fallback to ResNet18")
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 2)

model = model.to(device)

# timm 모델은 내부 구조가 조금 다를 수 있으나, 
# 학습 시 optimizer나 loss function은 동일하게 사용 가능합니다.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

Loading MobileViT Small Model via timm...
✅ MobileViT loaded via timm


mobilevit

In [11]:
# 주피터 노트북 셀에서 실행 시 앞에 !를 붙이세요
!pip install --upgrade torch torchvision

In [20]:
!python -m pip install --upgrade pip

In [12]:
import timm # 추가

# ================================
# 4. 모델 정의 (MobileViT Small via timm)
# ================================
print("Loading MobileViT Small Model using timm...")

# timm을 사용하면 num_classes를 직접 지정하여 마지막 레이어를 자동으로 맞춰줍니다.
# pretrained=True로 설정하면 ImageNet 가중치를 가져옵니다.
model = timm.create_model('mobilevit_s', pretrained=True, num_classes=2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
# timm 모델도 파라미터 구조는 표준 PyTorch 모델과 같습니다.
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

print("Model loaded successfully!")

Loading MobileViT Small Model using timm...
Model loaded successfully!


In [8]:
import torchvision
print(torchvision.__version__) 
# 0.13.0 이상이어야 함

0.21.0


In [13]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cuda


In [18]:
# ================================
# 4. 모델 정의 (timm을 사용하여 MobileViT 로드) ★ 수정된 부분 ★
# ================================
print("Loading MobileViT Small Model (via timm)...")

# timm을 사용하여 MobileViT 모델을 불러옵니다.
# num_classes=2로 설정하면 마지막 분류 레이어가 자동으로 이진 분류에 맞게 변경됩니다.
try:
    # timm이 설치되어 있다면 이 부분이 실행됩니다.
    model = timm.create_model('mobilevit_s', pretrained=True, num_classes=2)
    print("✅ MobileViT Small Model successfully loaded via timm.")
except Exception as e:
    # 혹시 timm 설치가 실패했거나 모델 이름이 잘못된 경우의 대체 모델 (ResNet18)
    print(f"timm 로드 실패: {e}")
    print("ResNet18로 대체 로드합니다. (권장: timm 설치 후 재시도)")
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

# 참고: 모델 저장 경로는 이미 이전 코드에서 `/workspace/user5`로 설정되었습니다.

Loading MobileViT Small Model (via timm)...
✅ MobileViT Small Model successfully loaded via timm.


In [21]:
import os
import json
import random
import numpy as np
import cv2 

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets
from tqdm import tqdm
import copy
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2 

# ★ timm 라이브러리 임포트 (버전 오류 해결용) ★
import timm 

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# ================================
# 3. Transform / Dataset / DataLoader
# ================================
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])

data_root = "/workspace/user4/cropped"

train_full_dataset = datasets.ImageFolder(os.path.join(data_root, "train"), transform=None) 
val_full_dataset = datasets.ImageFolder(os.path.join(data_root, "val"), transform=None) 

target_classes = ["anger", "happy", "panic", "sadness"]

def filter_and_remap_dataset(full_dataset, transform, target_classes): 
    new_samples = []
    
    for path, class_idx in full_dataset.samples:
        class_name = full_dataset.classes[class_idx]
        
        if class_name not in target_classes:
            continue
            
        # 2진 분류 라벨 할당 (Happy: 1, Others: 0)
        if class_name == "happy":
            binary_label = 1 
        else:
            binary_label = 0 
            
        new_samples.append((path, binary_label))
        
    class CustomBinaryDataset(Dataset):
        def __init__(self, samples, transform):
            self.samples = samples
            self.transform = transform
            
        def __len__(self):
            return len(self.samples)
            
        def __getitem__(self, idx):
            path, label = self.samples[idx]
            
            image = cv2.imread(path)
            if image is None:
                 raise FileNotFoundError(f"Image not found or invalid: {path}")

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            if self.transform is not None:
                augmented = self.transform(image=image)
                image = augmented['image'] 
            
            return image, torch.tensor(label, dtype=torch.long)

    print(f"Original {len(full_dataset)} samples reduced to {len(new_samples)}.")
    return CustomBinaryDataset(new_samples, transform) 


train_dataset = filter_and_remap_dataset(train_full_dataset, train_transform, target_classes) 
val_dataset = filter_and_remap_dataset(val_full_dataset, val_transform, target_classes) 


batch_size = 32
num_workers = 2 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

print("\n✅ 전처리 및 데이터 로드 완료.")

# ================================
# 4. 모델 정의 (timm 사용으로 오류 해결)
# ================================
print("Loading MobileViT Small Model (via timm)...")

# ★ 여기서 timm을 사용하여 모델을 불러옵니다. (기존 코드 대체) ★
# num_classes=2로 설정하면 마지막 레이어를 자동으로 맞춰줍니다.
try:
    model = timm.create_model('mobilevit_s', pretrained=True, num_classes=2)
except Exception as e:
    print(f"timm 로드 실패. 라이브러리가 설치되었는지 확인하세요. 에러: {e}")
    # 혹시 모를 상황을 대비한 예비책 (ResNet18)
    print("ResNet18로 대체 로드합니다.")
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)


# ================================
# 5. 학습 함수 (저장 경로 수정됨)
# ================================
# ★ 저장 경로 변수 설정 ★
SAVE_DIR = "/workspace/user5"
SAVE_PATH = os.path.join(SAVE_DIR, "best_mobilevit_happy_vs_others.pth")

# 폴더가 없으면 생성
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"Model will be saved to: {SAVE_PATH}")

def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        # ---- Train ----
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0

        with torch.inference_mode():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = val_corrects.double() / val_total
        print(f"Val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # best model 저장 (경로 수정됨)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            
            # ★ 여기서 지정한 경로에 저장합니다 ★
            torch.save(best_model_wts, SAVE_PATH)
            print(f">> best model updated and saved to {SAVE_PATH}")

    print(f"\nBest val Acc: {best_val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model


# ================================
# 6. 학습 실행
# ================================
num_epochs = 10
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs)


# ================================
# 7. 성능 지표
# ================================
model.eval()
all_labels = []
all_preds = []

with torch.inference_mode():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

acc  = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, pos_label=1)
rec  = recall_score(all_labels, all_preds, pos_label=1)
f1   = f1_score(all_labels, all_preds, pos_label=1)
cm   = confusion_matrix(all_labels, all_preds)

print("\n=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===")
print(f"Model: MobileViT Small")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall    : {rec:.4f}")
print(f"F1-score : {f1:.4f}")
print("\nConfusion Matrix (rows: true, cols: pred)")
print(cm)

print("\nClassification report:")
print(classification_report(all_labels, all_preds, target_names=["OTHERS", "HAPPY"]))

device: cuda
Original 9550 samples reduced to 9535.
Original 1151 samples reduced to 1149.

✅ 전처리 및 데이터 로드 완료.
Loading MobileViT Small Model (via timm)...
Model will be saved to: /workspace/user5/best_mobilevit_happy_vs_others.pth

Epoch 1/10
------------------------------


100%|██████████| 298/298 [00:45<00:00,  6.50it/s]

Train Loss: 0.2507 Acc: 0.9183


Val   Loss: 0.1172 Acc: 0.9634
>> best model updated and saved to /workspace/user5/best_mobilevit_happy_vs_others.pth

Epoch 2/10
------------------------------


100%|██████████| 298/298 [00:36<00:00,  8.19it/s]

...

Train Loss: 0.0123 Acc: 0.9962


Val   Loss: 0.1632 Acc: 0.9678

Epoch 10/10
------------------------------


100%|██████████| 298/298 [00:36<00:00,  8.22it/s]

Train Loss: 0.0089 Acc: 0.9974


Val   Loss: 0.2580 Acc: 0.9547

Best val Acc: 0.9730

=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===
Model: MobileViT Small
Accuracy : 0.9730
Precision: 0.9268
Recall    : 0.9732
F1-score : 0.9494

Confusion Matrix (rows: true, cols: pred)
[[827  23]
 [  8 291]]

Classification report:
              precision    recall  f1-score   support

      OTHERS       0.99      0.97      0.98       850
       HAPPY       0.93      0.97      0.95       299

    accuracy                           0.97      1149
   macro avg       0.96      0.97      0.97      1149
weighted avg       0.97      0.97      0.97      1149



In [24]:
!pip install tabulate

In [2]:
!pip install thop

  Using cached thop-0.1.1.post2209072238-py3-none-any.whl.metadata (2.7 kB)
Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)


In [7]:

import torch
import torch.nn as nn
import time
import statistics

# --- 선택적 모듈 존재 여부 확인 ---
has_timm = False
has_thop = False
try:
    import timm
    has_timm = True
except Exception:
    pass

try:
    from thop import profile
    has_thop = True
except Exception:
    pass

# torchvision은 폴백용으로만 필요
try:
    import torchvision.models as tv_models
except Exception:
    tv_models = None

# 1. 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ 사용 장치: {device}")
print("-" * 30)

# 2. 모델 로드: 우선 timm의 MobileViT-S 시도, 없으면 torchvision MobileNetV3-Small
MODEL_NAME = "MobileViT-S"
NUM_CLASSES = 2
print("⏳ 모델 로딩 중...")

model = None
used_backend = None

if has_timm:
    try:
        # timm으로 MobileViT-S
        model = timm.create_model('mobilevit_s', pretrained=True, num_classes=NUM_CLASSES)
        used_backend = "timm-MobileViT-S"
        print("✅ timm: MobileViT-S 로드 완료 (pretrained)")
    except Exception as e:
        print(f"⚠️ timm MobileViT-S 로드 실패: {e}")

if model is None and tv_models is not None:
    try:
        # torchvision 폴백: MobileNetV3-Small
        tv_model = tv_models.mobilenet_v3_small(weights=None)
        # 분류기 헤드 교체
        in_features = tv_model.classifier[-1].in_features
        tv_model.classifier[-1] = nn.Linear(in_features, NUM_CLASSES)
        model = tv_model
        used_backend = "torchvision-MobileNetV3-Small"
        print("✅ torchvision: MobileNetV3-Small 폴백 로드 (pretrained 없음)")
    except Exception as e:
        print(f"⚠️ torchvision MobileNetV3 로드 실패: {e}")

if model is None:
    raise RuntimeError("모델 로드에 실패했습니다. timm 설치 또는 torchvision 업데이트가 필요합니다.")

model = model.to(device)

# 3. 사용자 학습 가중치 로드 (선택)
save_path = "best_mobilevit_happy_vs_others.pth"
try:
    state_dict = torch.load(save_path, map_location=device)
    missing, unexpected = model.load_state_dict(state_dict, strict=False)
    print(f"✅ 최적 가중치 로드 완료: {save_path}")
    if missing:
        print(f"ℹ️ 누락된 키: {missing}")
    if unexpected:
        print(f"ℹ️ 예상치 못한 키: {unexpected}")
except FileNotFoundError:
    print(f"🚨 경고: '{save_path}' 파일을 찾을 수 없습니다. (테스트를 위해 기본/사전학습 가중치로 진행)")

model.eval()

# 4. Dummy Input (MobileViT 권장 256, 폴백도 동일 처리)
input_size = 256
dummy_input = torch.randn(1, 3, input_size, input_size).to(device)
print("✅ 모델 및 Dummy Input 준비 완료.")
print(f"입력 데이터 형태: {dummy_input.shape}")
print("-" * 30)

# 5. 파라미터 수 & FLOPs
params_m = sum(p.numel() for p in model.parameters()) / 1e6
flops_g = 0.0

if has_thop:
    try:
        macs, params = profile(model, inputs=(dummy_input,), verbose=False)
        flops_g = macs / 1e9 * 2  # FLOPs ≈ MACs * 2
        params_m = params / 1e6
        print(f"=== 모델 복잡도 측정 (thop) ===")
        print(f"파라미터 수 (Params): {params_m:.2f} M")
        print(f"연산량 (FLOPs): {flops_g:.2f} G")
        print("-" * 30)
    except Exception as e:
        print(f"⚠️ thop/FLOPs 계산 중 오류: {e}")
        print(f"➡️ 파라미터만 계산: {params_m:.2f} M")
        print("-" * 30)
else:
    print(f"➡️ thop 미설치: 파라미터만 계산 {params_m:.2f} M, FLOPs=0.00 G")
    print("-" * 30)

# 6. 추론 속도 측정
ITERATIONS = 100
print(f"=== 추론 속도 측정 시작 (Warm-up 10회, 측정 {ITERATIONS}회) ===")

with torch.no_grad():
    for _ in range(10):
        _ = model(dummy_input)

if device.type == 'cuda':
    torch.cuda.synchronize()

timings = []
if device.type == 'cuda':
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    with torch.no_grad():
        for _ in range(ITERATIONS):
            starter.record()
            _ = model(dummy_input)
            ender.record()
            torch.cuda.synchronize()
            timings.append(starter.elapsed_time(ender))  # ms
else:
    with torch.no_grad():
        for _ in range(ITERATIONS):
            t0 = time.time()
            _ = model(dummy_input)
            t1 = time.time()
            timings.append((t1 - t0) * 1000.0)

mean_ms = statistics.mean(timings)
std_ms = statistics.stdev(timings) if len(timings) > 1 else 0.0

# 7. 출력 (ASCII 테이블)
ACCURACY_REF = "MobileViT-S: ~78.4% (ImageNet Top-1, ref.)"  # 참고치

headers = ["백엔드/모델", "크기(px)", "정확도(참고)", f"속도 {device.type.upper()} (ms)", "파라미터(M)", "FLOPs(G)"]
row = [used_backend, str(input_size), ACCURACY_REF, f"{mean_ms:.2f} ± {std_ms:.2f}", f"{params_m:.2f}", f"{flops_g:.2f}"]

def render_table(headers, row):
    widths = [max(len(h), len(r)) for h, r in zip(headers, row)]
    def fmt(vals): return "| " + " | ".join(v.ljust(w) for v, w in zip(vals, widths)) + " |"
    sep = "+" + "+".join("-" * (w + 2) for w in widths) + "+"
    return "\n".join([sep, fmt(headers), sep, fmt(row), sep])

print("\n" + "=" * 85)
print("=== 성능 요약 ===")
print(render_table(headers, row))
print("=" * 85)


✅ 사용 장치: cuda
------------------------------
⏳ 모델 로딩 중...
✅ timm: MobileViT-S 로드 완료 (pretrained)
✅ 최적 가중치 로드 완료: best_mobilevit_happy_vs_others.pth
✅ 모델 및 Dummy Input 준비 완료.
입력 데이터 형태: torch.Size([1, 3, 256, 256])
------------------------------
=== 모델 복잡도 측정 (thop) ===
파라미터 수 (Params): 4.93 M
연산량 (FLOPs): 3.66 G
------------------------------
=== 추론 속도 측정 시작 (Warm-up 10회, 측정 100회) ===

=== 성능 요약 ===
+------------------+--------+--------------------------------------------+--------------+---------+----------+
| 백엔드/모델           | 크기(px) | 정확도(참고)                                    | 속도 CUDA (ms) | 파라미터(M) | FLOPs(G) |
+------------------+--------+--------------------------------------------+--------------+---------+----------+
| timm-MobileViT-S | 256    | MobileViT-S: ~78.4% (ImageNet Top-1, ref.) | 6.61 ± 0.40  | 4.93    | 3.66     |
+------------------+--------+--------------------------------------------+--------------+---------+----------+


seg코드

In [1]:

# -*- coding: utf-8 -*-
import os
import copy
import random
import numpy as np
from tqdm.auto import tqdm
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

# scikit-learn metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)

# MobileViT from timm
try:
    import timm
except ImportError as e:
    raise ImportError("timm가 설치되어 있지 않습니다. 먼저 `pip install timm`을 실행하세요.") from e


# -----------------------------
# 0) 재현성 설정(선택)
# -----------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # 아래 두 줄은 약간의 성능 저하가 있을 수 있습니다.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# -----------------------------
# 1) 경로 및 타깃 클래스 설정
# -----------------------------
train_dir = "/workspace/user4/segmented_output_bisenet/train"
val_dir   = "/workspace/user4/segmented_output_bisenet/val"

target_classes = ["anger", "happy", "panic", "sadness"]  # 요청하신 타깃 클래스

# -----------------------------
# 2) 디바이스 설정
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------------
# 3) 변환 정의
#    MobileViT는 기본적으로 256x256 입력을 많이 사용합니다.
#    ImageNet 평균/표준편차로 정규화합니다.
# -----------------------------
IMG_SIZE = 256
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.05, hue=0.02),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# -----------------------------
# 4) 전체 데이터 로드 (ImageFolder)
# -----------------------------
full_train = datasets.ImageFolder(train_dir)
full_val   = datasets.ImageFolder(val_dir)

print(f"Full train classes: {full_train.classes}")
print(f"Full val   classes: {full_val.classes}")

# -----------------------------
# 5) 요청하신 필터 + 이진 라벨 리맵핑 Dataset
#    (happy: 1, others: 0)
# -----------------------------
class BinaryFilteredDataset(Dataset):
    def __init__(self, full_dataset, transform, target_classes):
        self.transform = transform
        self.samples = []
        # full_dataset.samples: [(path, class_idx), ...]
        for path, class_idx in full_dataset.samples:
            class_name = full_dataset.classes[class_idx]
            if class_name not in target_classes:
                continue
            # 이진 라벨 매핑
            binary_label = 1 if class_name == "happy" else 0
            self.samples.append((path, binary_label))
        # 통계 출력
        total = len(self.samples)
        num_happy = sum(1 for _, y in self.samples if y == 1)
        num_others = total - num_happy
        print(f"[{type(self).__name__}] Total: {total} | HAPPY(1): {num_happy} | OTHERS(0): {num_others}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)


# 필터링된 데이터셋 생성
train_dataset = BinaryFilteredDataset(full_train, transform=train_transform, target_classes=target_classes)
val_dataset   = BinaryFilteredDataset(full_val,   transform=val_transform,   target_classes=target_classes)

# -----------------------------
# 6) DataLoader 준비
# -----------------------------
BATCH_SIZE = 32
NUM_WORKERS = 4

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

# -----------------------------
# 7) MobileViT 모델 정의
#    num_classes=2 (binary)
# -----------------------------
# 예: 'mobilevit_s', 'mobilevit_xs' 등 사용 가능. 여기선 small 사용.
model_name = "mobilevit_s"
model = timm.create_model(model_name, pretrained=True, num_classes=2)
model.to(device)

# -----------------------------
# 8) 손실함수/옵티마이저 정의
# -----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

# -----------------------------
# 9) 저장 경로 설정 (요청사항 반영)
# -----------------------------
SAVE_DIR = "/workspace/user5"
SAVE_PATH = os.path.join(SAVE_DIR, "best_mobilevit_happy_vs_others.pth")
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"Model will be saved to: {SAVE_PATH}")

# -----------------------------
# 10) 학습 루프 (요청하신 형태 + epoch=10)
# -----------------------------
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        # ---- Train ----
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels in tqdm(train_loader, leave=False):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / total if total > 0 else 0.0
        epoch_acc = (running_corrects.double() / total) if total > 0 else torch.tensor(0.0)
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0

        with torch.inference_mode():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                val_total += labels.size(0)

        val_loss = val_loss / val_total if val_total > 0 else 0.0
        val_acc = (val_corrects.double() / val_total) if val_total > 0 else torch.tensor(0.0)
        print(f"Val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # best model 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, SAVE_PATH)
            print(f">> best model updated and saved to {SAVE_PATH}")

    print(f"\nBest val Acc: {best_val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model

# -----------------------------
# 11) 학습 실행 (epoch=10)
# -----------------------------
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

# -----------------------------
# 12) 성능 평가 (요청하신 포맷)
# -----------------------------
model.eval()
all_labels = []
all_preds = []

with torch.inference_mode():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds  = np.array(all_preds)

acc  = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, pos_label=1)
rec  = recall_score(all_labels, all_preds, pos_label=1)
f1   = f1_score(all_labels, all_preds, pos_label=1)
cm   = confusion_matrix(all_labels, all_preds)

print("\n=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===")
print(f"Model: MobileViT Small")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")
print("\nConfusion Matrix (rows: true, cols: pred)")
print(cm)

print("\nClassification report:")
print(classification_report(all_labels, all_preds, target_names=["OTHERS", "HAPPY"]))


Using device: cuda
Full train classes: ['anger', 'happy', 'panic', 'sadness']
Full val   classes: ['anger', 'happy', 'panic', 'sadness']
[BinaryFilteredDataset] Total: 9535 | HAPPY(1): 5245 | OTHERS(0): 4290
[BinaryFilteredDataset] Total: 1149 | HAPPY(1): 299 | OTHERS(0): 850
Model will be saved to: /workspace/user5/best_mobilevit_happy_vs_others.pth

Epoch 1/10
------------------------------


  0%|          | 0/298 [00:00<?, ?it/s]

Train Loss: 0.1677 Acc: 0.9399
Val   Loss: 0.1219 Acc: 0.9652
>> best model updated and saved to /workspace/user5/best_mobilevit_happy_vs_others.pth

Epoch 2/10
------------------------------


  0%|          | 0/298 [00:00<?, ?it/s]

Train Loss: 0.0723 Acc: 0.9751
Val   Loss: 0.1965 Acc: 0.9373

Epoch 3/10
------------------------------


...

  0%|          | 0/298 [00:00<?, ?it/s]

Train Loss: 0.0186 Acc: 0.9932
Val   Loss: 0.1900 Acc: 0.9634

Epoch 10/10
------------------------------


  0%|          | 0/298 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fb9c4894e50>
Traceback (most recent call last):
  File "/home/user5/miniconda3/envs/tomas/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/home/user5/miniconda3/envs/tomas/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
  File "/home/user5/miniconda3/envs/tomas/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fb9c4894e50>
Traceback (most recent call last):
  File "/home/user5/miniconda3/envs/tomas/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/home/user5/miniconda3/envs/tomas/lib/python3

Train Loss: 0.0164 Acc: 0.9933
Val   Loss: 0.1605 Acc: 0.9687

Best val Acc: 0.9765

=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===
Model: MobileViT Small
Accuracy : 0.9765
Precision: 0.9359
Recall   : 0.9766
F1-score : 0.9558

Confusion Matrix (rows: true, cols: pred)
[[830  20]
 [  7 292]]

Classification report:
              precision    recall  f1-score   support

      OTHERS       0.99      0.98      0.98       850
       HAPPY       0.94      0.98      0.96       299

    accuracy                           0.98      1149
   macro avg       0.96      0.98      0.97      1149
weighted avg       0.98      0.98      0.98      1149



efficientformer 모델

In [3]:
# -*- coding: utf-8 -*-
import os
import copy
import random
import numpy as np
from tqdm.auto import tqdm
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

# scikit-learn metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)

# EfficientFormerV2 from timm
try:
    import timm
except ImportError as e:
    raise ImportError("timm이 설치되어 있지 않습니다. 먼저 `pip install timm`을 실행하세요.") from e


# -----------------------------
# 0) 재현성 설정(선택)
# -----------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        # 아래 두 줄은 약간의 성능 저하가 있을 수 있습니다.
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

# -----------------------------
# 1) 경로 및 타깃 클래스 설정
# -----------------------------
# 경로 설정은 사용자 환경에 맞게 유지합니다.
train_dir = "/workspace/user4/segmented_output_bisenet/train"
val_dir   = "/workspace/user4/segmented_output_bisenet/val"

target_classes = ["anger", "happy", "panic", "sadness"]  # 요청하신 타깃 클래스

# -----------------------------
# 2) 디바이스 설정
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------------
# 3) 변환 정의 (IMG_SIZE를 224로 수정)
#    EfficientFormerV2의 안정적인 동작을 위해 224x224로 설정합니다.
# -----------------------------
# 🚨 수정: IMG_SIZE를 256 -> 224로 변경하여 Attention 블록의 Shape 오류 방지
IMG_SIZE = 224
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.05, hue=0.02),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# -----------------------------
# 4) 전체 데이터 로드 (ImageFolder)
# -----------------------------
full_train = datasets.ImageFolder(train_dir)
full_val   = datasets.ImageFolder(val_dir)

print(f"Full train classes: {full_train.classes}")
print(f"Full val   classes: {full_val.classes}")

# -----------------------------
# 5) 요청하신 필터 + 이진 라벨 리맵핑 Dataset (happy: 1, others: 0)
# -----------------------------
class BinaryFilteredDataset(Dataset):
    def __init__(self, full_dataset, transform, target_classes):
        self.transform = transform
        self.samples = []
        # full_dataset.samples: [(path, class_idx), ...]
        for path, class_idx in full_dataset.samples:
            class_name = full_dataset.classes[class_idx]
            if class_name not in target_classes:
                continue
            # 이진 라벨 매핑
            binary_label = 1 if class_name == "happy" else 0
            self.samples.append((path, binary_label))
        # 통계 출력
        total = len(self.samples)
        num_happy = sum(1 for _, y in self.samples if y == 1)
        num_others = total - num_happy
        print(f"[{type(self).__name__}] Total: {total} | HAPPY(1): {num_happy} | OTHERS(0): {num_others}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)


# 필터링된 데이터셋 생성
train_dataset = BinaryFilteredDataset(full_train, transform=train_transform, target_classes=target_classes)
val_dataset   = BinaryFilteredDataset(full_val,   transform=val_transform,   target_classes=target_classes)

# -----------------------------
# 6) DataLoader 준비
# -----------------------------
BATCH_SIZE = 32
NUM_WORKERS = 4 # 환경에 따라 적절히 설정 (1-4 권장)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

# -----------------------------
# 7) EfficientFormerV2 모델 정의
#    num_classes=2 (binary)
# -----------------------------
model_name = "efficientformerv2_l"
# pretrained=True로 ImageNet 가중치 로드
model = timm.create_model(model_name, pretrained=True, num_classes=2)
model.to(device)
print(f"Using Model: {model_name} with num_classes=2") # 모델 사용 정보 출력

# -----------------------------
# 8) 손실함수/옵티마이저 정의
# -----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

# -----------------------------
# 9) 저장 경로 설정 (요청사항 반영)
# -----------------------------
SAVE_DIR = "/workspace/user5"
SAVE_PATH = os.path.join(SAVE_DIR, "best_efficientformerv2_happy_vs_others.pth")
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"Model will be saved to: {SAVE_PATH}")

# -----------------------------
# 10) 학습 루프 (epoch=10)
# -----------------------------
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        # ---- Train ----
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        # tqdm.auto.tqdm을 사용하여 진행 상황 표시
        for inputs, labels in tqdm(train_loader, desc=f"Train Epoch {epoch+1}"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / total if total > 0 else 0.0
        # .item()을 사용하여 텐서에서 Python 숫자로 변환
        epoch_acc = (running_corrects.double() / total).item() if total > 0 else 0.0
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0

        with torch.inference_mode():
            # tqdm.auto.tqdm을 사용하여 진행 상황 표시
            for inputs, labels in tqdm(val_loader, desc=f"Val   Epoch {epoch+1}"):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                val_total += labels.size(0)

        val_loss = val_loss / val_total if val_total > 0 else 0.0
        # .item()을 사용하여 텐서에서 Python 숫자로 변환
        val_acc = (val_corrects.double() / val_total).item() if val_total > 0 else 0.0
        print(f"Val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # best model 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, SAVE_PATH)
            print(f">> best model updated and saved to {SAVE_PATH}")

    print(f"\nBest val Acc: {best_val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model

# -----------------------------
# 11) 학습 실행 (epoch=10)
# -----------------------------
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

# -----------------------------
# 12) 성능 평가 (요청하신 포맷)
# -----------------------------
model.eval()
all_labels = []
all_preds = []

with torch.inference_mode():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds  = np.array(all_preds)

acc  = accuracy_score(all_labels, all_preds)
# pos_label=1: "happy" 클래스를 긍정(Positive)으로 간주
prec = precision_score(all_labels, all_preds, pos_label=1, zero_division=0)
rec  = recall_score(all_labels, all_preds, pos_label=1, zero_division=0)
f1   = f1_score(all_labels, all_preds, pos_label=1, zero_division=0)
cm   = confusion_matrix(all_labels, all_preds)

print("\n=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===")
print(f"Model: {model_name.upper()}")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")
print("\nConfusion Matrix (rows: true, cols: pred)")
print(cm)

print("\nClassification report:")
# target_names: 0 -> OTHERS, 1 -> HAPPY
print(classification_report(all_labels, all_preds, target_names=["OTHERS", "HAPPY"], zero_division=0))

Using device: cuda
Full train classes: ['anger', 'happy', 'panic', 'sadness']
Full val   classes: ['anger', 'happy', 'panic', 'sadness']
[BinaryFilteredDataset] Total: 9535 | HAPPY(1): 5245 | OTHERS(0): 4290
[BinaryFilteredDataset] Total: 1149 | HAPPY(1): 299 | OTHERS(0): 850
Using Model: efficientformerv2_l with num_classes=2
Model will be saved to: /workspace/user5/best_efficientformerv2_happy_vs_others.pth

Epoch 1/10
------------------------------


Train Epoch 1:   0%|          | 0/298 [00:00<?, ?it/s]

Train Loss: 0.1508 Acc: 0.9400


Val   Epoch 1:   0%|          | 0/36 [00:00<?, ?it/s]

Val   Loss: 0.4159 Acc: 0.9286
>> best model updated and saved to /workspace/user5/best_efficientformerv2_happy_vs_others.pth

Epoch 2/10
------------------------------


...

Val   Loss: 0.1372 Acc: 0.9713

Epoch 10/10
------------------------------


Train Epoch 10:   0%|          | 0/298 [00:00<?, ?it/s]

Train Loss: 0.0119 Acc: 0.9961


Val   Epoch 10:   0%|          | 0/36 [00:00<?, ?it/s]

Val   Loss: 0.1254 Acc: 0.9756
>> best model updated and saved to /workspace/user5/best_efficientformerv2_happy_vs_others.pth

Best val Acc: 0.9756

=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===
Model: EFFICIENTFORMERV2_L
Accuracy : 0.9756
Precision: 0.9302
Recall   : 0.9799
F1-score : 0.9544

Confusion Matrix (rows: true, cols: pred)
[[828  22]
 [  6 293]]

Classification report:
              precision    recall  f1-score   support

      OTHERS       0.99      0.97      0.98       850
       HAPPY       0.93      0.98      0.95       299

    accuracy                           0.98      1149
   macro avg       0.96      0.98      0.97      1149
weighted avg       0.98      0.98      0.98      1149

